In [ ]:
import numpy as np

def neural_gas(input_data, n_neurons, max_iterations=100, epsilon_start=0.5, epsilon_end=0.01):
    n_samples, n_features = input_data.shape

    # Initialize code vectors randomly from the input data
    code_vectors = input_data[np.random.choice(n_samples, n_neurons, replace=False)]

    # Initialize connections and lifetimes
    connections = np.zeros((n_neurons, n_neurons))
    lifetimes = np.zeros((n_neurons, n_neurons))
    for i in range(n_neurons):
        for j in range(i + 1, n_neurons):
            connections[i, j] = connections[j, i] = 1
            lifetimes[i, j] = lifetimes[j, i] = 10

    for iteration in range(max_iterations):
        epsilon = epsilon_start * (epsilon_end / epsilon_start) ** (iteration / (max_iterations - 1)) # epsilon scheduling using formula at the end of paper

        # Compute distances between each data point and all code vectors
        distances = np.array([np.linalg.norm(code_vectors - data, axis=1) for data in input_data])

        # Sort distances to find neighborhood ranks
        sorted_indices = np.argsort(distances, axis=1)
        neighborhood_ranks = np.argsort(sorted_indices, axis=1)

        # Update code vectors
        for i in range(n_neurons):
            neighbors = np.where(neighborhood_ranks == i)[0]
            if len(neighbors) > 0:
                for j, neighbor in enumerate(neighbors):
                    code_vectors[i] += epsilon * (input_data[neighbor] - code_vectors[i]) * np.exp(-j / len(neighbors))

        # Update connections and lifetimes
        for i in range(n_neurons):
            for j in range(n_neurons):
                if i != j:
                    age_factor = np.sum(np.exp(-np.abs(neighborhood_ranks[:, i] - neighborhood_ranks[:, j])))
                    connections[i, j] += age_factor
                    lifetimes[i, j] += age_factor

        # Decay lifetimes
        lifetimes *= 0.5

    return code_vectors, connections